## Загрузка библиотек и исходных данных

In [2]:
pip install laspy                                                               # Инсталируем пакет по работе с облаком точек las

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for laspy: filename=laspy-2.4.1-py3-none-any.whl size=68063 sha256=86058b6b68e38555bcd30dd76bb55e9cc79efba4dff21d95ff8a791ef1458e18
  Stored in directory: /root/.cache/pip/wheels/b7/84/b9/28a55f13245e29f4bee5274790ace34180da34cf230679fcf3
Successfully built laspy


In [3]:
import time, random, gdown, os                                                  # Системные инструменты
import numpy as np                                                              # Работа с массивами
import math
import laspy                                                                    # Библиотека для работы с точками las
import cv2

import matplotlib.pyplot as plt                                                 # Отрисовка графиков
from mpl_toolkits.mplot3d import Axes3D                                         # Matplotlib 3D графики
import seaborn as sns                                                           # Визуальные инструменты Seaborn
sns.set_style('darkgrid')
np.set_printoptions(suppress=True, linewidth=250)                               # Настройка печати Numpy

In [4]:
# Загрузка файла из облака на диск виртуальной машины colab
gdown.download('https://drive.google.com/uc?id=1nqY3MKkbz8-NzNy9NGPds4Z3yZ4m9tND', None, quiet=True)        # Путь к файлу pc_2021.las

'pc_2021.las'

In [6]:
filename = 'pc_2021.las'                                                        # las файл
filefolder = '/content/'                                                        # путь к файлу 

Файл LAS содержит информацию о положении (x, y, z) и других свойствах точек, таких как цвет, интенсивность и классификация. Файлы LAS являются стандартным форматом для обработки данных лазерного сканирования. 

Такой тип облака точек обозначают как 'unstructured D-D data'. В отличии от данных, имеющих определенную структуру и организованых в виде равномерной сетки узлов, которые содержат значения на определенных координатах, "unstructured D-D data" не имеют определенной структуры и могут быть представлены в виде набора точек, где каждая точка может иметь свои собственные координаты и значения.

## Перевод облака точек

In [7]:
las = laspy.read(f'{filefolder}{filename}')                                     # Считываем все данные из файла *.las в память

Удаление точек категории 2, 7 и 18

In [8]:
mask = (las.classification==2)|(las.classification==7)|(las.classification==18) # Маска с точками класса 2, 7 и 18
points = las[~mask]                                                             # Точки только 1 и 5 классов

In [9]:
coords = np.vstack((points.x, points.y, points.z)).transpose()                                                # массив координат XYZ
colors = np.vstack((points.red,points.green,points.blue,points.nir)).transpose() / 65280.                     # нормализированный массив информации о цвете

In [10]:
coords.shape, colors.shape

((1111300, 3), (1111300, 4))

Зададим размеры сетки вокселей

In [11]:
X_SIZE = 4000                                                                   
Y_SIZE = 4000 
Z_SIZE = math.ceil(points.z.max() - points.z.min())                             # Z = разность между максимальной и минимальной высотой точек
voxel_cloud = np.zeros((X_SIZE, Y_SIZE, Z_SIZE, 4))                             # пустой массив вокселей заданного размера, где 4 это 4 цвета точек
#voxel_grid = np.indices((X_SIZE, Y_SIZE, Z_SIZE, 4))                            # координатная сетка пространства заданной размерности и заданного размера, где 4 это 4 цвета точек

Считаем, в какой воксель попадает каждая точка из файла LAS

In [12]:
min_x = points.x.min()
min_y = points.y.min()
min_z = points.z.min()

for i in range(len(coords)):
    x, y, z = coords[i,0], coords[i,1], coords[i,2]
    color = colors[i]
    voxel_x = int((x - min_x) / X_SIZE)
    voxel_y = int((y - min_y) / Y_SIZE)
    voxel_z = int((z - min_z) / Z_SIZE)
    voxel_cloud[voxel_x, voxel_y, voxel_z] = color 
    #voxel_grid[voxel_x, voxel_y, voxel_z] = color

In [13]:
voxel_cloud.shape

(4000, 4000, 38, 4)

Сохраняем полученные воксели в файл формата VOX

In [14]:
voxel_cloud.tofile(f'{filefolder}pc_2021.vox')

## Нарезаем воксели на датасет

In [15]:
win_size = 256                                                                  # размер окна
hop =100                                                                       # шаг смещения окна
path_x = '/content/'
name_x = 'x_vox'

Функция 

In [16]:
def split_voksel(array, win_size, hop, path, name):
    '''Функция разбиения массива вокселей на меньшие массивы и их схранение на диске
    Param:
        array - входящий массив с вокселями 
        win_size -  размер окна
        hop -  шаг смещения окна
        path - путь где сохраняться файлы
        name - имя под которым сохраняться файлы файлов
    '''
    n = 0
    for y in range(0, array.shape[1] - win_size + 1, hop):                      # Цикл по координатам x и y
        for x in range(0, array.shape[0] - win_size + 1, hop):
            slice_x = slice(x, x + win_size)                                    # Определяем координаты срезов
            slice_y = slice(y, y + win_size)
            subarray = array[slice_x, slice_y, :, :]                            # Вырезаем подмассив
            filename = f'{path}{name}_{n}.npy'  
            #filename = f'{path}{name}_{x}_{y}.npy'                             # альтернативное именование файлов. переменная n будет не нужна
            np.save(filename, subarray)                                         # Сохраняем подмассивы в отдельный файл
            n += 1 

In [ ]:
split_voksel(voxel_cloud, win_size, hop, path_x, name_x)

# Проверяем как выглядят воксели

In [ ]:
'''
# Получение координат всех вокселей
x, y, z = np.indices(voxel_grid.shape)

# Получение массива цветов для каждого вокселя
colors = np.empty(voxel_cloud.shape, dtype=object)
colors[voxel_cloud] = 'blue'
colors[~voxel_cloud] = 'white'

# Создание фигуры и осей
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Отображение вокселей
ax.voxels(x, y, z, voxel_cloud, facecolors=colors)

# Настройка осей
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
'''